# Scrap do Campeonato Brasileiro de Futebol - 2019

<span>Obs.: instale as bibliotecas necessárias para utilização</span>

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import os
import sqlite3
from sqlalchemy import create_engine

In [ ]:
# Configuração para exibir todas as colunas do dataframe
pd.set_option('display.max_columns', None)

In [ ]:
# URL do site da CBF com os dados do campeonato
url = "https://www.cbf.com.br/futebol-brasileiro/competicoes/campeonato-brasileiro-serie-a"

In [ ]:
dados = requests.get(url)

In [ ]:
soup = BeautifulSoup(dados.content, 'html.parser')

In [ ]:
print(soup.title)

In [ ]:
table = soup.findAll('table')[0]

In [ ]:
colunas = []
for thead in table.findAll('thead'):
    #print(thead.getText())
    colunas.append(thead.getText())
    break

In [ ]:
linhas = []
for row in table.findAll('tbody'):
    #print(row)
    linha = []
    for tr in row.findAll('tr'):
        linha.append(tr)
        #print(len(linhas))
        if len(linha) % 2:
            #print(tr.getText())
            linhas.append(tr.getText().replace('º', ''))
    break

In [ ]:
linhas = [i.replace('\n', ',') for i in linhas]

In [ ]:
arr = np.asarray(linhas)

In [ ]:
arr

In [ ]:
# salva os dados em um arquivo temporário
temp_file = 'temp_rodada.csv'
with open(temp_file, "w") as f:
    f.write("\n".join("".join(map(str, x)) for x in (arr)))

In [ ]:
colunas = ['A','B','Posicao','Evolucao','C','Clube','D','Pontos','Jogos','Vitorias','Empates','Derrotas','GolsPro','GolsContra','SaldoGols','Amarelos','Vermelhos','Aproveitamento','E','F','G','H','I','J','K','L','M']
colunas

In [ ]:
df = pd.read_csv(temp_file, sep=',', names=colunas, encoding='ISO-8859-1', na_filter=True, index_col=0)

In [ ]:
df

In [ ]:
df.dropna(axis=1, inplace=True)

In [ ]:
df = df[['Posicao','Clube','Pontos','Jogos','Vitorias','Empates','Derrotas','GolsPro','GolsContra','SaldoGols','Amarelos','Vermelhos','Aproveitamento']]

In [ ]:
df

In [ ]:
today = datetime.date.today()

In [ ]:
today

In [ ]:
df.insert(0, 'Data', today)

In [ ]:
df.columns

In [ ]:
df

In [ ]:
df.info()

### Grava o arquivo na pasta do app web

In [ ]:
filename = 'data/rodada_' + str(today) + '.csv'

In [ ]:
df.to_csv(filename, encoding='ISO-8859-1')

In [ ]:
temp_file

In [ ]:
if os.path.exists(temp_file):
    os.remove(temp_file)

## Criar DB

In [ ]:
engine = create_engine('sqlite:///data/rodada.db', echo=False)
df.to_sql('brasileirao', con=engine, if_exists='append', index=True)
print(engine.execute("SELECT * FROM brasileirao").fetchall())

### SELECT no DB

In [ ]:
print(engine.execute("SELECT * FROM brasileirao where clube = 'São Paulo - SP'").fetchall())

# FIM